# Scénario 1 : Exploration des Collections de Presse de Gallica sur les Épidémies au XIXe Siècle

**Chercheuse (fictive) :** Jeanne Dupont, historienne, spécialiste du XIXe siècle.

**Phase du projet :** Tout début. Elle a une thématique (représentation des épidémies dans la presse française) mais besoin d'aide pour évaluer la faisabilité et constituer un premier corpus de sources primaires.

**Objectif de cet accompagnement (simulé via ce notebook) :**
1. Identifier les titres de presse pertinents et le volume d'articles/fascicules potentiels dans Gallica traitant des épidémies durant une période ciblée du XIXe siècle.
2. Obtenir une liste structurée de ces documents avec leurs métadonnées de base (titre, périodique, date, lien Gallica).
3. Avoir une première idée de la répartition chronologique de ces publications.
4. Comprendre comment accéder concrètement aux documents numérisés (images, OCR si disponible) via l'API IIIF.

---
**Méthodologie :**
* Utilisation de l'API SRU (Search/Retrieve via URL) de Gallica pour la recherche par mots-clés et filtres.
* Ciblage d'une période spécifique du XIXe siècle pour la démonstration.
* Extraction et structuration des métadonnées en utilisant Python et la bibliothèque Pandas.
* Analyse exploratoire simple (comptages, distribution temporelle).
* Introduction à l'API IIIF pour l'accès au contenu.

## 0. Importation des bibliothèques nécessaires

Avant de commencer, nous importons les bibliothèques Python qui seront utiles pour ce travail :
* `requests` pour effectuer les requêtes HTTP vers l'API Gallica.
* `xml.etree.ElementTree` pour parser les réponses XML de l'API SRU.
* `pandas` pour manipuler et analyser les données structurées.
* `matplotlib.pyplot` pour créer des visualisations graphiques.
* `time` pour introduire des délais entre les requêtes API et ne pas surcharger les serveurs.
* `urllib.parse` pour encoder correctement les URLs.
* `json` pour manipuler les données JSON (notamment pour les manifestes IIIF).
* `os` pour la gestion des dossiers de sortie.

In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import time
import urllib.parse
import json
import os

# Configuration de l'affichage pour Pandas et Matplotlib
pd.set_option('display.max_colwidth', 200)
plt.style.use('seaborn-v0_8-whitegrid') # Utilisation d'un style pour les graphiques

## 1. Définition de la Stratégie de Recherche dans Gallica

### 1.1. Paramètres de la recherche

Pour cette démonstration, nous allons cibler :
* **Mots-clés :** Une liste de termes relatifs aux épidémies.
* **Période :** Nous allons nous concentrer sur une période illustrative, par exemple **1830-1870**. Cette période couvre plusieurs vagues importantes de choléra en France et une activité journalistique en développement.
* **Type de document :** Fascicules de périodiques (`dc.type all "fascicule"`).

In [ ]:
KEYWORDS = ["choléra", "épidémie", "contagion", "typhus", "variole"]
START_YEAR = 1830
END_YEAR = 1870
GALLICA_SRU_ENDPOINT = "http://gallica.bnf.fr/SRU"
MAX_RECORDS_PER_QUERY = 50 
MAX_TOTAL_RECORDS_TO_FETCH_PER_KEYWORD = 150 
POLITENESS_DELAY = 1 # Secondes entre les requêtes

### 1.2. Création des dossiers de sortie
Si les dossiers n'existent pas, nous les créons.

In [ ]:
output_data_folder = 'output_data'
plots_folder = os.path.join(output_data_folder, 'plots')

if not os.path.exists(output_data_folder):
    os.makedirs(output_data_folder)
if not os.path.exists(plots_folder):
    os.makedirs(plots_folder)

print(f"Les données de sortie seront sauvegardées dans : '{output_data_folder}'")
print(f"Les graphiques seront sauvegardés dans : '{plots_folder}'")

## 2. Interaction avec l'API SRU de Gallica

### 2.1. Fonction pour interroger l'API SRU

Nous allons créer une fonction qui construit la requête CQL (Contextual Query Language) et interroge l'API Gallica SRU.
La fonction gérera la pagination pour récupérer plusieurs lots de résultats.

In [ ]:
def fetch_gallica_sru(keyword, start_year, end_year, start_record=1, max_records=MAX_RECORDS_PER_QUERY):
    """
    Interroge l'API SRU de Gallica pour un mot-clé donné et une période.
    Retourne le contenu XML de la réponse.
    """
    # Construction de la requête CQL
    # Note: gallica all "[keyword]" recherche le mot-clé dans tous les champs indexés par Gallica.
    # ocr.text all "[keyword]" chercherait uniquement dans le texte OCR, mais peut être plus lent/moins complet.
    query = f'gallica all "{keyword}" and dc.type all "fascicule" and (gallicapublication_date >= "{start_year}" and gallicapublication_date <= "{end_year}")'
    
    params = {
        'operation': 'searchRetrieve',
        'version': '1.2',
        'query': query,
        'maximumRecords': max_records,
        'startRecord': start_record,
        'collapsing': 'false' # Pour éviter le regroupement des résultats par pertinence (ex: par titre de périodique)
    }
    
    try:
        response = requests.get(GALLICA_SRU_ENDPOINT, params=params, timeout=30) # Timeout de 30s
        response.raise_for_status()  # Lève une exception pour les codes d'erreur HTTP 4xx/5xx
        return response.content
    except requests.exceptions.RequestException as e:
        print(f"Erreur de requête pour '{keyword}', start_record {start_record}: {e}")
        return None

### 2.2. Test rapide de la fonction `Workspace_gallica_sru`
Testons avec un mot-clé et peu de résultats pour vérifier la connexion et le format de la réponse.

In [ ]:
test_xml_content = fetch_gallica_sru("choléra", 1832, 1832, max_records=5)
if test_xml_content:
    print("Réponse XML reçue (premiers 500 caractères) :\n", test_xml_content[:500].decode('utf-8'))
else:
    print("Aucune réponse reçue du test.")

## 3. Parsing XML et Extraction des Métadonnées

### 3.1. Fonctions pour parser le XML et extraire les informations

L'API SRU retourne du XML qui utilise des éléments Dublin Core (préfixe `dc:`).
Nous devons aussi gérer les namespaces XML.

In [ ]:
# Namespaces couramment utilisés dans les réponses SRU de Gallica
NAMESPACES = {
    'sru': 'http://www.loc.gov/zing/srw/',
    'g': 'http://www.loc.gov/zing/srw/', # Parfois 'g' est utilisé comme alias pour sru
    'dc': 'http://purl.org/dc/elements/1.1/',
    'oai_dc': 'http://www.openarchives.org/OAI/2.0/oai_dc/',
    'gallica': 'http://gallica.bnf.fr/ns/gallica#',
    'xlink': 'http://www.w3.org/1999/xlink'
}

def parse_sru_response(xml_content, keyword_searched):
    """
    Parse le contenu XML d'une réponse SRU et extrait les métadonnées des documents.
    Retourne une liste de dictionnaires, chaque dictionnaire représentant un document.
    """
    records = []
    if not xml_content:
        return records, 0 # Retourne aussi 0 pour total_results

    try:
        root = ET.fromstring(xml_content)
        
        # Nombre total de résultats estimé par Gallica 
        # S'il n'y a pas de résultat, la balise 'numberOfRecords' peut manquer
        num_results_element = root.find('.//sru:numberOfRecords', NAMESPACES)
        if num_results_element is None: # Essayer avec 'g' si 'sru' ne marche pas
             num_results_element = root.find('.//g:numberOfRecords', NAMESPACES)

        total_results = int(num_results_element.text) if num_results_element is not None and num_results_element.text.isdigit() else 0
        
        for record_element in root.findall('.//sru:recordData/oai_dc:dc', NAMESPACES):
            data = {}
            data['keyword_searched'] = keyword_searched # Ajout du mot-clé ayant permis de trouver le document

            # Extraction des champs Dublin Core. On prend le premier élément trouvé pour chaque champ.
            # Certains champs peuvent être multiples, mais pour cette exploration on simplifie.
            for field in ['title', 'creator', 'publisher', 'date', 'type', 'format', 'language', 'source', 'relation', 'description']:
                element = record_element.find(f'dc:{field}', NAMESPACES)
                data[f'dc_{field}'] = element.text.strip() if element is not None and element.text else None
            
            # L'identifiant ARK est important
            identifier_elements = record_element.findall('dc:identifier', NAMESPACES)
            ark_identifier = None
            for elem in identifier_elements:
                if elem.text and 'ark:/' in elem.text:
                    ark_identifier = elem.text.strip()
                    break # On prend le premier ARK trouvé
            data['ark'] = ark_identifier
            data['gallica_url'] = ark_identifier # L'ARK est aussi l'URL Gallica

            records.append(data)
            
        return records, total_results
    
    except ET.ParseError as e:
        print(f"Erreur de parsing XML: {e}")
        # print("Contenu XML problématique (premiers 1000 caractères): ", xml_content[:1000])
        return [], 0
    except Exception as e:
        print(f"Erreur inattendue lors du parsing: {e}")
        return [], 0

### 3.2. Collecte des données pour tous les mots-clés

Nous allons maintenant boucler sur nos mots-clés, interroger Gallica, et stocker toutes les métadonnées.
Pour limiter la durée de cette démonstration, nous allons chercher au maximum `MAX_TOTAL_RECORDS_TO_FETCH_PER_KEYWORD` par mot-clé.

In [ ]:
all_metadata = []
total_gallica_estimate = {} # Estimation du nombre total de résultats par Gallica pour chaque mot-clé

print(f"Début de la collecte des métadonnées depuis Gallica ({START_YEAR}-{END_YEAR})...\n" + "="*30)

for keyword in KEYWORDS:
    print(f"\nRecherche pour le mot-clé : '{keyword}'")
    keyword_metadata = []
    current_start_record = 1
    fetched_for_keyword = 0
    estimated_total_for_keyword = 0
    
    # Récupérer la première page pour connaître le nombre total estimé de résultats
    xml_content_first_page = fetch_gallica_sru(keyword, START_YEAR, END_YEAR, start_record=1, max_records=1)
    if xml_content_first_page:
        _, estimated_total_for_keyword = parse_sru_response(xml_content_first_page, keyword)
        total_gallica_estimate[keyword] = estimated_total_for_keyword
        print(f"Gallica estime {estimated_total_for_keyword} résultats pour '{keyword}'.")
        if estimated_total_for_keyword == 0:
            print(f"Aucun résultat pour '{keyword}'. Passage au suivant.")
            time.sleep(POLITENESS_DELAY) # Pause même si pas de résultat
            continue
    else:
        print(f"Impossible de récupérer l'estimation pour '{keyword}'. Passage au suivant.")
        time.sleep(POLITENESS_DELAY)
        continue

    while fetched_for_keyword < MAX_TOTAL_RECORDS_TO_FETCH_PER_KEYWORD:
        print(f"  Récupération des résultats {current_start_record} à {current_start_record + MAX_RECORDS_PER_QUERY -1 }...")
        xml_content = fetch_gallica_sru(keyword, START_YEAR, END_YEAR, start_record=current_start_record)
        
        if xml_content:
            new_records, _ = parse_sru_response(xml_content, keyword) # Le total ici est celui de la requête, pas global
            if not new_records: # Plus de résultats ou erreur
                print(f"  Plus de résultats ou erreur pour '{keyword}' à partir de {current_start_record}.")
                break
            
            keyword_metadata.extend(new_records)
            fetched_for_keyword += len(new_records)
            current_start_record += MAX_RECORDS_PER_QUERY

            # Si on a dépassé le nombre total estimé de résultats ou si on a récupéré moins de résultats que demandé (fin des résultats)
            if current_start_record > estimated_total_for_keyword or len(new_records) < MAX_RECORDS_PER_QUERY :
                 print(f"  Fin des résultats pour '{keyword}' (ou limite estimée atteinte).")
                 break
        else:
            print(f"  Erreur lors de la récupération pour '{keyword}', arrêt pour ce mot-clé.")
            break
            
        time.sleep(POLITENESS_DELAY) # Soyons polis avec l'API
    
    all_metadata.extend(keyword_metadata)
    print(f"Total de {len(keyword_metadata)} notices récupérées pour '{keyword}'.")

print("\n" + "="*30 + "\nCollecte terminée.")
print(f"Nombre total de notices (brutes, potentiellement avec doublons) : {len(all_metadata)}")
print("Estimations de Gallica pour le nombre total de résultats :")
for kw, count in total_gallica_estimate.items():
    print(f"  - '{kw}': {count}")

## 4. Structuration des Données avec Pandas

### 4.1. Création du DataFrame

Convertissons la liste de dictionnaires en DataFrame Pandas pour une manipulation plus aisée.

In [ ]:
df_gallica = pd.DataFrame(all_metadata)

print(f"\nLe DataFrame contient {df_gallica.shape[0]} lignes et {df_gallica.shape[1]} colonnes.")
print("Aperçu des premières lignes du DataFrame :")
display(df_gallica.head())
print("\nInformations sur le DataFrame :")
df_gallica.info()

### 4.2. Nettoyage et Prétraitement

* **Doublons :** Un même document (ARK) peut être retourné pour plusieurs mots-clés. Nous allons les dédoublonner.
* **Dates :** Le champ `dc_date` peut contenir des formats variés. Nous allons essayer de l'uniformiser et d'extraire l'année.
* **Titre du périodique :** Souvent dans `dc_source` ou `dc_relation`. Nous allons tenter de le normaliser.

In [ ]:
# Dédoublonnage basé sur l'ARK
if 'ark' in df_gallica.columns and not df_gallica['ark'].isnull().all():
    print(f"\nNombre de lignes avant dédoublonnage : {len(df_gallica)}")
    # Pour garder une trace des mots-clés qui ont mené à un document, on peut les agréger
    # Cependant, pour cette première exploration, on garde juste la première occurrence
    df_gallica_unique = df_gallica.drop_duplicates(subset=['ark'], keep='first').copy() # .copy() pour éviter SettingWithCopyWarning
    print(f"Nombre de lignes après dédoublonnage sur 'ark' : {len(df_gallica_unique)}")
else:
    print("\nColonne 'ark' non trouvée ou vide, impossible de dédoublonner.")
    df_gallica_unique = df_gallica.copy()

# Traitement des dates
# dc_date peut être une année, une date complète (YYYY-MM-DD), ou une période (YYYY-YYYY)
# Pour simplifier, nous extrayons l'année. On prend les 4 premiers chiffres s'ils ressemblent à une année.
def extract_year(date_str):
    if pd.isna(date_str):
        return None
    # Essayer de matcher une année (4 chiffres) au début de la chaîne
    import re
    match = re.match(r'^(\d{4})', str(date_str))
    if match:
        year = int(match.group(1))
        # Vérifier si l'année est dans une plage plausible (ex: 1500-2025)
        if START_YEAR <= year <= END_YEAR + 20 : # un peu de marge
            return year
    return None

if 'dc_date' in df_gallica_unique.columns:
    df_gallica_unique['year'] = df_gallica_unique['dc_date'].apply(extract_year)
    # Filtrer les années qui ne sont pas dans notre période d'étude après extraction
    df_gallica_unique = df_gallica_unique[df_gallica_unique['year'].between(START_YEAR, END_YEAR, inclusive='both')]
    print(f"\nNombre de lignes après filtrage par année ({START_YEAR}-{END_YEAR}) : {len(df_gallica_unique)}")
else:
    print("\nColonne 'dc_date' non trouvée pour l'extraction de l'année.")
    df_gallica_unique['year'] = None # Créer la colonne pour éviter les erreurs futures

# Titre du périodique (souvent dans dc:source ou dc:relation)
# Exemple : "Le Charivari. 26 septembre 1848" -> "Le Charivari"
# Pour une analyse plus poussée, il faudrait une regex plus fine ou un lien vers une notice de périodique.
# Pour l'instant, on utilise dc:source si disponible, sinon dc:relation. dc:creator est aussi une piste.
def extract_journal_title(row):
    if pd.notna(row.get('dc_source')): # dc:source est souvent le titre du périodique
        # Tenter de nettoyer un peu, ex: enlever la date si présente
        # Ceci est très basique, pour une vraie analyse il faudrait des listes d'autorité ou des regex plus robustes
        title_candidate = str(row['dc_source']).split('.')[0].split('(')[0].strip()
        if len(title_candidate) > 3 : # Eviter les abréviations trop courtes
             return title_candidate
    if pd.notna(row.get('dc_relation')): # dc:relation peut aussi contenir le titre
         title_candidate = str(row['dc_relation']).split('.')[0].split('(')[0].strip()
         if len(title_candidate) > 3 :
             return title_candidate
    if pd.notna(row.get('dc_creator')): # Parfois le créateur est le journal
         title_candidate = str(row['dc_creator']).split('.')[0].split('(')[0].strip()
         if len(title_candidate) > 3 :
             return title_candidate
    return "Non identifié"

if 'dc_source' in df_gallica_unique.columns:
    df_gallica_unique['journal_title'] = df_gallica_unique.apply(extract_journal_title, axis=1)
else:
    df_gallica_unique['journal_title'] = "Non identifié"


print("\nAperçu du DataFrame nettoyé :")
display(df_gallica_unique[['ark', 'dc_title', 'journal_title', 'year', 'keyword_searched', 'gallica_url']].head())

### 4.3. Sauvegarde des métadonnées structurées

Sauvegardons ce DataFrame nettoyé en CSV pour une réutilisation future ou pour la chercheuse.
Nous sauvegardons un échantillon pour ne pas surcharger le dépôt Git de démonstration.

In [ ]:
sample_size_csv = min(500, len(df_gallica_unique)) # Sauvegarder au max 500 lignes
df_gallica_sample_csv = df_gallica_unique.sample(n=sample_size_csv, random_state=42) if len(df_gallica_unique) > sample_size_csv else df_gallica_unique

csv_path = os.path.join(output_data_folder, f'gallica_epidemies_{START_YEAR}-{END_YEAR}_metadata_sample.csv')
df_gallica_sample_csv.to_csv(csv_path, index=False)
print(f"\nUn échantillon de {len(df_gallica_sample_csv)} métadonnées nettoyées a été sauvegardé dans : {csv_path}")

# Si on voulait sauvegarder tout le dataframe (attention à la taille pour un repo Git)
# full_csv_path = os.path.join(output_data_folder, f'gallica_epidemies_{START_YEAR}-{END_YEAR}_metadata_FULL.csv')
# df_gallica_unique.to_csv(full_csv_path, index=False)
# print(f"Toutes les {len(df_gallica_unique)} métadonnées nettoyées ont été sauvegardées dans : {full_csv_path}")

## 5. Analyse Exploratoire Initiale

### 5.1. Statistiques descriptives

Quelques chiffres clés pour donner une première idée du corpus potentiel à Dr. Moreau.

In [ ]:
if not df_gallica_unique.empty:
    total_unique_docs = len(df_gallica_unique)
    total_unique_journals = df_gallica_unique['journal_title'].nunique()
    
    print(f"\nStatistiques descriptives du corpus potentiel ({START_YEAR}-{END_YEAR}) :")
    print(f"  - Nombre total de fascicules uniques identifiés : {total_unique_docs}")
    print(f"  - Nombre de titres de périodiques uniques : {total_unique_journals}")

    if total_unique_journals > 0 and total_unique_journals != 1 and "Non identifié" in df_gallica_unique['journal_title'].value_counts().index: # Eviter l'erreur si 'Non identifié' est le seul.
      print(f"  - Nombre de fascicules avec titre de périodique 'Non identifié': {df_gallica_unique['journal_title'].value_counts()['Non identifié']}")
    
    print("\nTop 10 des périodiques les plus fréquents (hors 'Non identifié') :")
    top_journals = df_gallica_unique[df_gallica_unique['journal_title'] != "Non identifié"]['journal_title'].value_counts().nlargest(10)
    display(top_journals)
else:
    print("\nLe DataFrame est vide. Aucune analyse statistique possible.")

### 5.2. Analyse temporelle

Visualisons la distribution du nombre de fascicules par année.

In [ ]:
if not df_gallica_unique.empty and 'year' in df_gallica_unique.columns and df_gallica_unique['year'].notna().any():
    docs_per_year = df_gallica_unique['year'].value_counts().sort_index()
    
    plt.figure(figsize=(15, 7))
    docs_per_year.plot(kind='bar', color='skyblue')
    plt.title(f'Nombre de fascicules relatifs aux épidémies par année ({START_YEAR}-{END_YEAR}) dans Gallica', fontsize=15)
    plt.xlabel('Année', fontsize=12)
    plt.ylabel('Nombre de fascicules', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout() # Ajuste automatiquement les paramètres pour donner un bon agencement
    
    # Sauvegarde du graphique
    plot_path = os.path.join(plots_folder, f'distribution_temporelle_{START_YEAR}-{END_YEAR}.png')
    plt.savefig(plot_path)
    print(f"\nGraphique de la distribution temporelle sauvegardé dans : {plot_path}")
    
    plt.show()
else:
    print("\nPas de données d'année valides pour générer la distribution temporelle.")

### 5.3. Interprétation (pour Dr. Moreau)

* Le volume de documents potentiels semble [**important/modéré/faible** - à remplir en fonction des résultats réels].
* La distribution temporelle montre des pics d'intérêt pour le sujet des épidémies autour des années [**années des pics**], ce qui correspond plausiblement à [**événements historiques connus, ex: vagues de choléra**].
* Les périodiques comme [**Noms des top journaux**] semblent être des sources particulièrement riches.
* Il est important de noter que la recherche par mots-clés a ses limites (qualité de l'OCR, polysémie des termes). Une exploration plus fine, titre par titre, ou avec des techniques de TAL plus avancées, serait nécessaire pour affiner le corpus.

## 6. Pistes pour l'Accès au Contenu via l'API IIIF

### 6.1. Introduction à IIIF (International Image Interoperability Framework)

IIIF est un ensemble de standards qui permet aux bibliothèques, archives et musées de diffuser des images et des contenus audiovisuels de manière standardisée et interopérable. Gallica utilise IIIF.

Pour chaque document (ARK), on peut généralement accéder à un "manifeste IIIF". C'est un fichier JSON qui décrit le document numérique (sa structure, ses pages, les liens vers les images, parfois vers l'OCR, etc.).

### 6.2. Récupération et Analyse d'un Manifeste IIIF (Exemple)

Prenons un exemple d'ARK de notre corpus pour voir comment récupérer son manifeste IIIF.

In [ ]:
if not df_gallica_unique.empty and 'ark' in df_gallica_unique.columns:
    # Sélectionner un échantillon avec un ARK valide
    sample_doc_for_iiif_df = df_gallica_unique[df_gallica_unique['ark'].notna()]
    if not sample_doc_for_iiif_df.empty:
        sample_doc_for_iiif = sample_doc_for_iiif_df.sample(1).iloc[0]
        ark_example = sample_doc_for_iiif['ark']
        
        # L'ARK est de la forme "ark:/12148/btv1b10500001g". Pour l'API IIIF, on enlève le "ark:/".
        ark_id_for_iiif = ark_example.split('/')[-1] # Prend la partie après le dernier '/'
        
        manifest_url = f"https://gallica.bnf.fr/iiif/ark:/12148/{ark_id_for_iiif}/manifest.json"
        print(f"\nExemple de récupération du manifeste IIIF pour l'ARK : {ark_example}")
        print(f"URL du manifeste : {manifest_url}")
        
        try:
            response = requests.get(manifest_url, timeout=15)
            response.raise_for_status()
            manifest_data = response.json()
            
            # Sauvegarde du manifeste exemple
            manifest_filename = os.path.join(output_data_folder, f'sample_iiif_manifest_{ark_id_for_iiif}.json')
            with open(manifest_filename, 'w', encoding='utf-8') as f:
                json.dump(manifest_data, f, ensure_ascii=False, indent=4)
            print(f"Manifeste sauvegardé dans : {manifest_filename}")
            
            # Afficher quelques informations clés du manifeste
            print(f"\nLabel du manifeste : {manifest_data.get('label')}")
            
            if 'sequences' in manifest_data and len(manifest_data['sequences']) > 0:
                num_canvases = len(manifest_data['sequences'][0].get('canvases', []))
                print(f"Nombre de 'canvases' (pages/vues) dans la première séquence : {num_canvases}")

                if num_canvases > 0:
                    first_canvas = manifest_data['sequences'][0]['canvases'][0]
                    if 'images' in first_canvas and len(first_canvas['images']) > 0:
                        first_image_url = first_canvas['images'][0]['resource']['@id']
                        print(f"URL de la première image (qualité par défaut) : {first_image_url}")

            # Chercher les liens vers l'OCR (souvent en ALTO XML via 'seeAlso' ou 'rendering')
            ocr_links = []
            if 'seeAlso' in manifest_data:
                for item in manifest_data['seeAlso']:
                    if isinstance(item, dict) and item.get('format') == 'application/xml+alto':
                        ocr_links.append(item.get('@id'))
            # Parfois dans 'rendering' au niveau de la séquence ou du canvas
            if 'sequences' in manifest_data and manifest_data['sequences']:
                for rendering_item in manifest_data['sequences'][0].get('rendering', []):
                     if isinstance(rendering_item, dict) and "alto" in rendering_item.get('format', "").lower():
                          ocr_links.append(rendering_item.get('@id'))

            if ocr_links:
                print(f"Liens potentiels vers l'OCR (ALTO XML) trouvés : {ocr_links}")
            else:
                print("Aucun lien direct vers un fichier OCR ALTO trouvé dans les sections 'seeAlso' ou 'rendering' de ce manifeste.")
                
        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de la récupération du manifeste IIIF : {e}")
        except json.JSONDecodeError:
            print("Erreur lors du décodage du JSON du manifeste.")
        except Exception as e:
            print(f"Erreur inattendue avec le manifeste IIIF : {e}")
    else:
        print("\nAucun document avec ARK valide trouvé dans l'échantillon pour l'exemple IIIF.")
else:
    print("\nDataFrame vide ou sans ARK, impossible de récupérer un exemple de manifeste IIIF.")

### 6.3. Conclusion pour Jeanne Dupont (Accès au contenu)

L'API IIIF permet donc d'accéder :
* Aux **images** des pages du document (avec la possibilité de spécifier la taille, la rotation, la région, etc.).
* Aux **fichiers OCR** (quand ils existent et sont référencés), souvent au format ALTO XML. Ces fichiers contiennent le texte reconnu et la position des mots sur la page. Leur traitement permettrait des analyses textuelles plus poussées (ce qui pourrait être l'objet d'un accompagnement ultérieur).

## 7. Synthèse et Prochaines Étapes (pour Jeanne Dupont)

Ce premier travail d'exploration a permis de :
1.  Confirmer la présence de nombreuses sources potentielles dans Gallica pour votre étude sur les épidémies dans la presse du XIXe siècle (durant la période {START_YEAR}-{END_YEAR}).
2.  Fournir une première liste de métadonnées (échantillon dans `output_data/gallica_epidemies_{START_YEAR}-{END_YEAR}_metadata_sample.csv`) qui pourra servir de base à l'affinage du corpus.
3.  Identifier des pics d'activité rédactionnelle et des titres de presse clés.
4.  Montrer comment accéder aux documents numérisés eux-mêmes via IIIF, ouvrant la voie à une consultation directe ou à des traitements automatisés de l'OCR.

**Prochaines étapes suggérées pour votre recherche :**
* **Affiner la sélection des mots-clés :** Tester des synonymes, des expressions plus spécifiques, ou des noms de maladies moins courants.
* **Explorer les titres de presse identifiés :** Parcourir systématiquement certains numéros des journaux les plus prolifiques.
* **Considérer une période plus large ou différente** si nécessaire, en adaptant les paramètres de ce notebook.
* **Récupération et analyse de l'OCR :** Si l'analyse textuelle est un objectif, un travail spécifique sur la récupération et le traitement des fichiers ALTO XML sera nécessaire.
* **Vérification manuelle :** Toujours croiser les résultats automatiques avec une vérification manuelle d'échantillons pour évaluer la pertinence réelle des documents.

N'hésitez pas si vous avez d'autres questions ou si vous souhaitez approfondir certains aspects !

---
**Considérations sur la réutilisation des données de Gallica :**

Les contenus de Gallica sont, pour la plupart, dans le domaine public ou sous des licences permettant une large réutilisation, notamment pour la recherche.
Il est toutefois recommandé de consulter les conditions d'utilisation spécifiques sur le site de Gallica et de la BnF.
Pour un usage intensif des API, la BnF propose un portail dédié `api.bnf.fr` avec des clés d'accès qui peuvent être nécessaires pour éviter des limitations.

